In [ ]:
from sklearn import svm
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from packages.Utility.Gabor import GaborFilterBanks, GaborParams, RotatedGaborParams
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Layer, Dense
from keras.models import Model
from sklearn.externals import joblib
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.spatial import distance
from packages.Utility.SVM.SVM import *
from packages.Utility.SVM.Kernel import *
from packages.Utility.CNN.Callbacks import SaveModel

# Memuat Dataset

In [ ]:
dataset = np.load('dataset/bin/digit 32x32 50k/dataset.npy')
folds = np.load('dataset/bin/digit 32x32 50k/folds.npy')
labels = np.load('dataset/bin/digit 32x32 50k/target.npy')

# K-fold

In [ ]:
current_fold = 0
train_idx = folds[current_fold, 0]
test_idx = folds[current_fold, 1]

# Mendapatkan Dataset Untuk Training dan Testing

In [ ]:
X_train = np.array([i[0] for i in dataset[train_idx]]).reshape(dataset[train_idx].shape[0], 32, 32, 1)
Y_train = np.array([i[2] for i in dataset[train_idx]])
X_test = np.array([i[0] for i in dataset[test_idx]])
Y_test = np.array([i[2] for i in dataset[test_idx]])

# Pra-Training

In [ ]:
save_model = SaveModel(base_dir = 'trained/digit 32x32 50k LN1', X_val = X_train, Y_val = Y_train)

# CNN
Compile CNN pada umumnya

In [ ]:
model = Sequential()

c1 = Conv2D(name='c1', filters = 6, kernel_initializer = GaborFilterBanks(RotatedGaborParams(lambd = 0.5, psi = 1.57, sigma = 2, gamma = 0.75)),
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu', 
            input_shape = (32, 32, 1))
c2 = Conv2D(name='c2', filters = 16, kernel_initializer = GaborFilterBanks(RotatedGaborParams(lambd = 0.5, psi = 1.57, sigma = 2, gamma = 0.75)), 
            kernel_size = 5, strides = 1, padding = 'valid', activation = 'relu')
m1 = MaxPool2D(pool_size = 2)
m2 = MaxPool2D(pool_size = 2)
f1 = Flatten(name = 'flatten')
model.add(c1)
model.add(m1)
model.add(c2)
model.add(m2)
model.add(f1)
model.add(Dense(120, activation = 'sigmoid', name = 'd1'))
model.add(Dense(84, activation = 'sigmoid', name = 'd2'))
model.add(Dense(10, activation = 'softmax', name = 'd3'))

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, verbose = 1, batch_size = 128, callbacks=[save_model])

In [ ]:
X_test = np.array([i[0] for i in dataset[fold[0, 1]]])
Y_test_cnn = np.array([i[2] for i in dataset[fold[0, 1]]])
Y_test_svm = np.array([i[1] for i in dataset[fold[0, 1]]])

## Menghitung Akurasi CNN
### Akurasi data training

In [ ]:
predicted = model.predict(X_train)
y_pred = np.argmax(predicted, axis = 1)
y_true = dataset[fold[0, 0], 1].astype('int')

print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

### Akurasi data testing

In [ ]:
predicted = model.predict(X_test)
y_pred = np.argmax(predicted, axis = 1)
y_true = Y_test_svm.astype('int')

print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

# Melakukan Pelatihan dengan SVM
data dari Flatten layer dilatih pada SVM dan
diuji akurasinya

In [ ]:
feature_model = Model(
    inputs = model.input,
    outputs = model.get_layer('f1').output
)

features_train = feature_model.predict(X_train)

# cls = SVM(C = 100, tol = 0.01)
# cls.compile(max_passes = 2, max_iter = 5)
cls = svm.SVC(kernel = 'linear', max_iter = 50000, C = 100, decision_function_shape='ovo', verbose = True)
cls.fit(features_train, dataset[fold[0, 0], 1].astype('int'))
# joblib.dump(cls, 'trained/trained_svm_digit.sav')

In [ ]:
cls.support_

## Akurasi SVM
### Data Training

In [ ]:
# cls = joblib.load('trained/trained_svm_digit.sav')
y_pred = cls.predict(features_train)
y_true = dataset[fold[0, 0], 1].astype('int')

print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

### Data Testing

In [ ]:
features_test = feature_model.predict(X_test)

y_pred = cls.predict(features_test)
y_true = dataset[fold[0, 1], 1].astype('int')

print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

# Retrieval Dengan SVM sebagai klasifikasi

Kueri acak

In [ ]:
index_kueri = np.random.randint(X_test.shape[0])
kueri = X_test[index_kueri]

### Mendapatkan fitur dari data training

In [ ]:
train_feature = feature_model.predict(X_train)

## Menggabungkan dengan dataset training

In [ ]:
collection = np.append(dataset[fold[0, 0]], train_feature, axis = 1)

### Klasifikasi kueri

In [ ]:
# cls = joblib.load('trained/trained_svm_digit.sav')
ftr = feature_model.predict(np.array([kueri]))
pred = cls.predict(ftr)

### Mendapatkan daftar citra sesuai hasil klasifikasi

In [ ]:
box = collection[np.where(collection[:, 1] == pred[0])]

### Menghitung jarak

In [ ]:
jarak = []

for i in range(box.shape[0]):
    jarak.append([box[i, 3], distance.euclidean(box[i, 4:], ftr)])
    
jarak.sort(key = lambda x: x[1])    

### Menampilkan hasil

In [ ]:
print('Kelas kueri : ', dataset[fold[0, 1]][index_kueri, 1])
print('Prediksi : ', pred[0])

fig = plt.figure(figsize = (20, 20))

## menampilkn query
y = fig.add_subplot(5, 5, 1)
y.imshow(
    cv.cvtColor(cv.imread(dataset[fold[0, 1]][index_kueri, 3]), cv.COLOR_BGR2RGB)
)
plt.title('Kueri')
y.axes.get_xaxis().set_visible(False)
y.axes.get_yaxis().set_visible(False)
    
for i in range(1, 20):
    y = fig.add_subplot(5, 5, i + 1)
    y.imshow(cv.cvtColor(cv.imread(jarak[i][0]), cv.COLOR_BGR2RGB))
    plt.title('Jarak : ' + str(jarak[i][1]))
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)